# Парсер 2 часть.
__Функционал:__ Переход по ссылке на страницу игрока, сбор нужных данных о нём. Таким образом сбор всех нужных данных о всех игроках.

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pickle

from tqdm import tqdm
import concurrent.futures

In [2]:
URL_base = "https://www.transfermarkt.com/"

Открытие файла с ссылками на страницы игроков и описание заголовков для подключения.

In [3]:
with open('players_url.data', 'rb') as filehandle:
    players_url = pickle.load(filehandle)

In [4]:
name_columns = ["name", "age", "position", "height", "value", "national_matches", "national_goals",
                "matches20", "goals20", "assists20", "yellow20", "double_yellows20", "red20",
                "matches21", "goals21", "assists21", "yellow21", "double_yellows21", "red21",
                "matches22", "goals22", "assists22", "yellow22", "double_yellows22", "red22"]

In [5]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
}
TIMEOUT = 10


Сбор основной статистики об игре с его профиля.

In [6]:
def base_stat(url, number): # статистика с основной страницы профиля
    r = requests.get(URL_base + url + "/profil/spieler/" + number, headers={'User-Agent': 'Custom'})
    soup = bs(r.text, "html.parser")
    try:
        # возраст
        age = soup.select("[itemprop='birthDate']")[0]
        age = int(age.text.strip()[-3:-1])
    
        # позиция
        position = soup.select(".detail-position__position")[0]
        position = position.text.strip()
    
        # рост
        height = soup.select("[itemprop='height']")[0]
        height = float(height.text.strip()[:-2].replace(",", "."))
    
        # стоимость
        value = soup.select(".data-header__market-value-wrapper")[0]
        value = value.text.split()[0].replace(",", ".")
        mult = value[-1]
        value = float(value[1:-1]) * 1000 if mult == "m" else value[1:-1]
    except Exception as ex:
        print(ex, "base", url)
        age = 0
        position = 0
        height = 0
        value = 0
    # количество матчей и голов за сборную
    try:
        national_team = soup.select(".data-header__content--highlight")
        national_matches = int(national_team[0].text.strip())
        national_goals = int(national_team[1].text.strip())
    except Exception as ex:
        print(ex, "national", url)
        national_matches = 0
        national_goals = 0
        
    return [age, position, height, value, national_matches, national_goals]

Сбор подробной статистики об игроке(голы, матчи, ассисты за определенный год)

In [7]:
def adv_stat(url, number, year): # подробная статистика за сезон
    try:
        r = requests.get(URL_base + url + "/leistungsdaten/spieler/" + number + "/plus/0?saison=" + year, headers={'User-Agent': 'Custom'})
        soup = bs(r.text, "html.parser")
    
        # матчи голы ассисты желтые карточки красные карточки за сезон 
        statistic = soup.select(".rechts ~ td + .zentriert")
        matches = int(statistic[0].text) if statistic[0].text.isnumeric() else 0
        goals = int(statistic[1].text) if statistic[1].text.isnumeric() else 0
        assists = int(statistic[2].text) if statistic[2].text.isnumeric() else 0
        yellow = int(statistic[3].text) if statistic[3].text.isnumeric() else 0
        double_yellow = int(statistic[4].text) if statistic[4].text.isnumeric() else 0
        red = int(statistic[5].text) if statistic[5].text.isnumeric() else 0
    except Exception as ex:
        print(ex, "adv", url)
        matches = 0
        goals = 0
        assists = 0
        yellow = 0
        double_yellow = 0
        red = 0
        
    return [matches, goals, assists, yellow, double_yellow, red]

Функция, реализующая сбор базовой статистики игрока и подробной статистики за последние три года.

In [8]:
def player(url):
    number = url.split("/")[-1]
    player_base_url = url.split("/")[1]
    
    params = [player_base_url]
    
    params.extend(base_stat(player_base_url, number))
    
    for year in range(2020, 2023):
        params.extend(adv_stat(player_base_url, number, str(year)))
    
    """with open('df_football.csv', 'a', newline='') as f_object:  
    # Pass the CSV  file object to the writer() function
        writer_object = writer(f_object)
        # Result - a writer object
        # Pass the data in the list as an argument into the writerow() function
        writer_object.writerow(params)  
        # Close the file object
        f_object.close()"""
    return params
    

Создание датафрейма для данных

In [9]:
df = pd.DataFrame(columns=name_columns)

Проход циклом по всем игрокам из списка и сбор данных о них. Ускорение процесса с помощью многопоточности.

In [10]:
kek = []
CONNECTIONS = 10

with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
    future_to_url = (executor.submit(player, url)for url in players_url)
    for future in tqdm(concurrent.futures.as_completed(future_to_url), total=len(players_url)):
        data = future.result()
        kek.append(data)

  8%|██████▎                                                                        | 102/1266 [02:42<24:02,  1.24s/it]

list index out of range national diego-moreno


 10%|████████                                                                       | 129/1266 [03:22<26:18,  1.39s/it]

list index out of range national diego-moreno


 12%|█████████▌                                                                     | 153/1266 [03:54<22:20,  1.20s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████████▍| 1256/1266 [28:00<00:07,  1.38it/s]

list index out of range national amilton


 99%|█████████████████████████████████████████████████████████████████████████████▍| 1257/1266 [28:02<00:09,  1.04s/it]

list index out of range national douglas-tanque


100%|██████████████████████████████████████████████████████████████████████████████| 1266/1266 [28:10<00:00,  1.34s/it]


In [11]:
df = pd.DataFrame(kek, columns=name_columns)

In [17]:
df = df[df.sum(axis=1) != 0]

C:\Users\persi\AppData\Local\Temp/ipykernel_9448/497139982.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df[df.sum(axis=1) != 0]


Запись полученных данных в файл.

In [18]:
df.to_csv("main_data.csv", index=False)

In [20]:
df

,name,age,position,height,value,national_matches,national_goals,matches20,goals20,assists20,...,assists21,yellow21,double_yellows21,red21,matches22,goals22,assists22,yellow22,double_yellows22,red22
0,martin-odegaard,24,Attacking Midfield,1.78,80000.0,49,2,29,2,2,...,5,4,0,0,45,15,8,5,0,0
1,erling-haaland,22,Centre-Forward,1.95,170000.0,23,21,41,41,12,...,8,3,0,0,52,52,9,5,0,0
2,bukayo-saka,21,Right Winger,1.78,110000.0,26,8,46,7,10,...,7,6,0,0,48,15,11,7,0,0
3,enzo-fernandez,22,Defensive Midfield,1.78,85000.0,12,2,40,2,5,...,7,3,0,0,51,4,9,11,0,0
4,declan-rice,24,Defensive Midfield,1.88,80000.0,41,3,35,2,1,...,4,11,0,0,50,5,4,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,abdulsamed-damlu,23,Goalkeeper,1.91,450,0,0,13,0,1,...,0,0,0,22,21,0,5,0,0,36
1262,emre-demir,19,Attacking Midfield,1.75,1500.0,12,1,16,2,1,...,1,1,0,0,4,1,0,1,0,0
1263,halil-akbunar,29,Right Winger,1.67,3000.0,2,0,42,10,14,...,7,6,0,1,34,4,7,4,0,0
1264,amilton,33,Left Winger,1.72,450,0,0,39,5,5,...,0,6,1,0,25,2,4,6,0,0
